In [1]:
######################################################
# INSULIN INDEX FROM BELL KJ THESES AND FOODSTRUCT
# Why? Because these are sources with verified values
######################################################

import pandas as pd
import argparse

argparser = argparse.ArgumentParser()
argparser.add_argument("--bell_kj_path", type=str, default="../data/raw/Bell_KJ_insulin_index.csv")
argparser.add_argument("--foodstruct_path", type=str, default="../data/raw/Foodstruct_insulin_index.csv")
argparser.add_argument("--name_col", type=str, default="Insulin Index Food Name")
argparser.add_argument("--output_path", type=str, default="../data/processed/insulin_index.csv")
args = argparser.parse_args()

def main(args):

    df_1 = pd.read_csv(args.bell_kj_path)
    df_1.drop(columns=["Weight (g/MJ)", "Food Test"], inplace=True)
    df_1.rename(columns={"FII (%)": "Insulin Index"}, inplace=True)
    
    df_2 = pd.read_csv(args.Foodstruct_insulin_index)
    df_2.rename(columns={"Food" : args.name_col}, inplace=True)

    df = pd.concat([df_1, df_2], axis=0).sort_values(by="Insulin Index")

    df = df[~df[args.name_col].duplicated(keep=False)].reset_index(drop=True)
    df[args.name_col] = df[args.name_col].str.replace('"', '', regex=False)
    df.to_csv(args.output_path, index=False)

if __name__ == "__main__":
    main(args)

In [4]:
######################################################################
# INSULIN INDEX FROM KAGGLE DATASET
# Why? Because it has a lot of data (more than Bell KJ & Foodstruct)
######################################################################

# PROBLEM: I have to merge the data with the main dataset, this is not implemented properly yet

import pandas as pd
import argparse

argparser = argparse.ArgumentParser()
argparser.add_argument("--main_data_path", type=str, default="../data/final/merged_rewe_fdc_insulin_time.csv")
argparser.add_argument("--kaggle_insulin_path", type=str, default="../data/raw/kaggle_satiety_dataset/satietyindex.csv")

argparser.add_argument("--output_path", type=str, default="../data/final/merged_rewe_fdc_insulin_time.csv")
args = argparser.parse_args()

def main(args):
  df = pd.read_csv(args.main_data_path)
  df_insulin = pd.read_csv(args.kaggle_insulin_path)

  df_insulin.sort_values(by="Insulin Index")
  df_insulin.rename(columns=
            {
              "Insulin Index": "'Non Nutrient Data.Insulin Index", 
              "Insulin Index Food Name": "Non Nutrient Data.Insulin Index Food Name",
              "'Non Nutrient Data.Insulin Index": "Non Nutrient Data.Insulin Index",
              }, 
  inplace=True)

  df_insulin.to_csv(args.output_path, index=False)


  # correct for missing time values
  df['Non Nutrient Data.Preparation Time'].fillna(df['Non Nutrient Data.Preparation Time'].mean(), inplace=True)
  df.to_csv(args.output_path, index=False)

if __name__ == "__main__":
    main(args)